In [1]:
import json
import requests
from typing import Any, Dict, List,Optional, Literal
from dataclasses import dataclass
from pydantic import BaseModel

In [2]:
{
  "id": "xyz",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "Let me call a tool.",
        "tool_calls": [
          {
            "id": "tool1",
            "type": "function",
            "function": {
              "name": "get_weather",
              "arguments": "{\"location\": \"London\"}"
            }
          }
        ]
      }
    }
  ],
  "created": 123456789,
  "model": "gpt-4",
  "object": "chat.completion"
}


{'id': 'xyz',
 'choices': [{'index': 0,
   'message': {'role': 'assistant',
    'content': 'Let me call a tool.',
    'tool_calls': [{'id': 'tool1',
      'type': 'function',
      'function': {'name': 'get_weather',
       'arguments': '{"location": "London"}'}}]}}],
 'created': 123456789,
 'model': 'gpt-4',
 'object': 'chat.completion'}

In [3]:
class ChatCompletionToolCallFunction(BaseModel):
    """The function that the model called"""
    arguments: str  # A JSON string of the arguments to call the function with
    name: str       # The name of the function to call

class ChatCompletionToolCall(BaseModel):
    """A tool call generated by the model"""
    id: str
    function: ChatCompletionToolCallFunction
    type: Literal["function"]

class ChatCompletionMessage(BaseModel):
    """A chat completion message generated by the model"""
    content: Optional[str] = None
    role: Literal["assistant"]
    tool_calls: Optional[List[ChatCompletionToolCall]] = None

class ChatCompletionChoice(BaseModel):
    """A chat completion choice"""
    finish_reason: Optional[str] = None
    index: int
    message: ChatCompletionMessage

class ChatCompletion(BaseModel):
    """Represents a chat completion response"""
    id: str
    choices: List[ChatCompletionChoice]
    created: int
    model: str
    object: Literal["chat.completion"]


In [4]:
class HTTPClient:
    """Simplified version of OpenAI's HTTP client"""
    
    def __init__(self, api_key: str, base_url: str):
        self.api_key = api_key
        self.base_url = base_url
    
    def post(self, endpoint: str, json_data: Dict[str, Any]) -> Dict[str, Any]:
        """Make HTTP POST request"""
        headers = {
            "Authorization": f"Bearer {self.api_key}",
            "Content-Type": "application/json",
            "User-Agent": "OpenAI/Python"
        }
        
        url = f"{self.base_url}{endpoint}"
        print(f"INTERNAL: POST {url}")
        print(f"INTERNAL: Headers: {list(headers.keys())}")
        
        response = requests.post(url, headers=headers, json=json_data)
        
        if response.status_code != 200:
            raise Exception(f"HTTP {response.status_code}: {response.text}")
        
        return response.json()


In [5]:
class Completions:
    """Handles the /chat/completions endpoint"""
    
    def __init__(self, http_client: HTTPClient):
        self._client = http_client
    
    def create(
        self,
        *,
        messages: List[Dict[str, Any]],
        model: str,
        tools: Optional[List[Dict[str, Any]]] = None,
        tool_choice: Optional[str] = None,
        **kwargs
    ) -> ChatCompletion:
        """Create a chat completion"""
        
        print("INTERNAL: Building request payload...")
        
        # 1. Build request body
        body = {
            "messages": messages,
            "model": model,
            **kwargs
        }
        
        if tools is not None:
            body["tools"] = tools
        
        if tool_choice is not None:
            body["tool_choice"] = tool_choice
        
        print(f"INTERNAL: Request body keys: {list(body.keys())}")
        
        # 2. Make HTTP request
        print("INTERNAL: Making HTTP request...")
        raw_response = self._client.post("/chat/completions", body)
        
        # 3. Parse response into typed objects
        print("INTERNAL: Parsing response into ChatCompletion object...")
        return self._parse_response(raw_response)
    
    def _parse_response(self, raw_response: Dict[str, Any]) -> ChatCompletion:
        """Parse raw JSON response into ChatCompletion object"""
        
        print("INTERNAL: Parsing choices array...")
        choices = []
        
        for choice_data in raw_response["choices"]:
            message_data = choice_data["message"]
            
            # Parse tool_calls if present
            tool_calls = None
            if "tool_calls" in message_data and message_data["tool_calls"]:
                print(f"INTERNAL: Found {len(message_data['tool_calls'])} tool calls")
                tool_calls = []
                
                for tc_data in message_data["tool_calls"]:
                    print(f"INTERNAL: Parsing tool call: {tc_data['function']['name']}")
                    
                    # Create function object
                    function = ChatCompletionToolCallFunction(
                        name=tc_data["function"]["name"],
                        arguments=tc_data["function"]["arguments"]  # Keep as JSON string!
                    )
                    
                    # Create tool call object
                    tool_call = ChatCompletionToolCall(
                        id=tc_data["id"],
                        type=tc_data["type"],
                        function=function
                    )
                    
                    tool_calls.append(tool_call)
            
            # Create message object
            message = ChatCompletionMessage(
                role=message_data["role"],
                content=message_data.get("content"),
                tool_calls=tool_calls
            )
            
            # Create choice object
            choice = ChatCompletionChoice(
                index=choice_data["index"],
                message=message,
                finish_reason=choice_data.get("finish_reason")
            )
            
            choices.append(choice)
        
        # Create main completion object
        completion = ChatCompletion(
            id=raw_response["id"],
            choices=choices,
            created=raw_response["created"],
            model=raw_response["model"],
            object=raw_response["object"]
        )
        
        print("INTERNAL: ChatCompletion object created successfully")
        return completion


In [6]:
class Chat:
    """Chat resource"""
    
    def __init__(self, http_client: HTTPClient):
        self.completions = Completions(http_client)

In [7]:
class OpenAI:
    """Main OpenAI client"""
    
    def __init__(self, api_key: str = None, base_url: str = "https://api.openai.com/v1"):
        if api_key is None:
            import os
            api_key = os.getenv("OPENAI_API_KEY", "dummy")
        
        self._http_client = HTTPClient(api_key, base_url)
        self.chat = Chat(self._http_client)

In [8]:

def demo_internal_parsing():
    """Show how OpenAI library internally parses tool_calls"""
    
    print("OPENAI LIBRARY INTERNAL PARSING DEMO")
    print("="*50)
    
    # Mock raw response from API (what HTTP client receives)
    raw_api_response = {
        "id": "chatcmpl-123",
        "object": "chat.completion",
        "created": 1677652288,
        "model": "gpt-4",
        "choices": [
            {
                "index": 0,
                "message": {
                    "role": "assistant",
                    "content": None,
                    "tool_calls": [
                        {
                            "id": "call_abc123",
                            "type": "function",
                            "function": {
                                "name": "get_weather",
                                "arguments": '{"city": "Paris", "units": "celsius"}'
                            }
                        },
                        {
                            "id": "call_def456", 
                            "type": "function",
                            "function": {
                                "name": "calculate",
                                "arguments": '{"expression": "15 + 25"}'
                            }
                        }
                    ]
                },
                "finish_reason": "tool_calls"
            }
        ]
    }
    
    print("1. Raw API Response (JSON):")
    print(json.dumps(raw_api_response, indent=2))
    
    # Show internal parsing
    completions = Completions(None)  # No HTTP client needed for parsing demo
    completion = completions._parse_response(raw_api_response)
    
    print("\n2. Parsed into Python objects:")
    print(f"Completion ID: {completion.id}")
    print(f"Model: {completion.model}")
    print(f"Choices count: {len(completion.choices)}")
    
    message = completion.choices[0].message
    print(f"Message role: {message.role}")
    print(f"Message content: {message.content}")
    print(f"Tool calls count: {len(message.tool_calls) if message.tool_calls else 0}")
    
    # Show tool_calls access
    if message.tool_calls:
        print("\n3. Accessing tool_calls:")
        for i, tool_call in enumerate(message.tool_calls):
            print(f"\nTool Call {i+1}:")
            print(f"  tool_call.id = '{tool_call.id}'")
            print(f"  tool_call.type = '{tool_call.type}'")
            print(f"  tool_call.function.name = '{tool_call.function.name}'")
            print(f"  tool_call.function.arguments = '{tool_call.function.arguments}'")
            print(f"  type(tool_call.function.arguments) = {type(tool_call.function.arguments)}")
            
            # Show JSON parsing
            args = json.loads(tool_call.function.arguments)
            print(f"  json.loads(arguments) = {args}")
            print(f"  type(parsed_args) = {type(args)}")

In [9]:
demo_internal_parsing()

OPENAI LIBRARY INTERNAL PARSING DEMO
1. Raw API Response (JSON):
{
  "id": "chatcmpl-123",
  "object": "chat.completion",
  "created": 1677652288,
  "model": "gpt-4",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": null,
        "tool_calls": [
          {
            "id": "call_abc123",
            "type": "function",
            "function": {
              "name": "get_weather",
              "arguments": "{\"city\": \"Paris\", \"units\": \"celsius\"}"
            }
          },
          {
            "id": "call_def456",
            "type": "function",
            "function": {
              "name": "calculate",
              "arguments": "{\"expression\": \"15 + 25\"}"
            }
          }
        ]
      },
      "finish_reason": "tool_calls"
    }
  ]
}
INTERNAL: Parsing choices array...
INTERNAL: Found 2 tool calls
INTERNAL: Parsing tool call: get_weather
INTERNAL: Parsing tool call: calculate
INTERNAL: ChatComplet

In [10]:
def demo_with_actual_client():
    """Demo using the recreated OpenAI client"""
    
    print("\n" + "="*60)
    print("DEMO WITH RECREATED OPENAI CLIENT")
    print("="*60)
    
    # Use our recreated client (pointing to Ollama for demo)
    client = OpenAI(
        api_key="ollama",
        base_url="http://localhost:11434/v1"
    )
    
    tools = [
        {
            "type": "function",
            "function": {
                "name": "get_weather",
                "description": "Get weather",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "city": {"type": "string"}
                    },
                    "required": ["city"]
                }
            }
        }
    ]
    
    try:
        # This shows the complete internal flow
        completion = client.chat.completions.create(
            model="llama3.2:1b",
            messages=[
                {"role": "user", "content": "What's the weather in Tokyo?"}
            ],
            tools=tools
        )
        
        print(f"\nResult type: {type(completion)}")
        print(f"Message: {completion.choices[0].message.content}")
        
        if completion.choices[0].message.tool_calls:
            for tc in completion.choices[0].message.tool_calls:
                print(f"Tool call: {tc.function.name}({tc.function.arguments})")
    
    except Exception as e:
        print(f"Error (expected if Ollama not running): {e}")


In [11]:
# Show with actual client
demo_with_actual_client()


DEMO WITH RECREATED OPENAI CLIENT
INTERNAL: Building request payload...
INTERNAL: Request body keys: ['messages', 'model', 'tools']
INTERNAL: Making HTTP request...
INTERNAL: POST http://localhost:11434/v1/chat/completions
INTERNAL: Headers: ['Authorization', 'Content-Type', 'User-Agent']
INTERNAL: Parsing response into ChatCompletion object...
INTERNAL: Parsing choices array...
INTERNAL: Found 1 tool calls
INTERNAL: Parsing tool call: get_weather
INTERNAL: ChatCompletion object created successfully

Result type: <class '__main__.ChatCompletion'>
Message: 
Tool call: get_weather({"city":"Tokyo"})


In [12]:
def show_key_implementation_details():
    """Show the key implementation details"""
    
    print("\n" + "="*60)
    print("KEY IMPLEMENTATION DETAILS")
    print("="*60)
    
    print("""
1. PYDANTIC MODELS:
   - OpenAI uses Pydantic for type validation
   - ChatCompletionToolCall, ChatCompletionMessage, etc.
   - Automatic JSON parsing and validation

2. ARGUMENTS HANDLING:
   - arguments field is ALWAYS stored as string
   - No automatic JSON parsing by the library
   - You must call json.loads() yourself

3. OBJECT HIERARCHY:
   ChatCompletion
   └── choices: List[ChatCompletionChoice]
       └── message: ChatCompletionMessage
           └── tool_calls: List[ChatCompletionToolCall]
               └── function: ChatCompletionToolCallFunction
                   ├── name: str
                   └── arguments: str  # JSON string!

4. PARSING FLOW:
   Raw JSON → Pydantic models → Python objects
   
5. THE CRITICAL POINT:
   tool_call.function.arguments is intentionally kept as a string
   This allows the library to be agnostic about argument types
    """)

In [13]:
# Show implementation details
show_key_implementation_details()


KEY IMPLEMENTATION DETAILS

1. PYDANTIC MODELS:
   - OpenAI uses Pydantic for type validation
   - ChatCompletionToolCall, ChatCompletionMessage, etc.
   - Automatic JSON parsing and validation

2. ARGUMENTS HANDLING:
   - arguments field is ALWAYS stored as string
   - No automatic JSON parsing by the library
   - You must call json.loads() yourself

3. OBJECT HIERARCHY:
   ChatCompletion
   └── choices: List[ChatCompletionChoice]
       └── message: ChatCompletionMessage
           └── tool_calls: List[ChatCompletionToolCall]
               └── function: ChatCompletionToolCallFunction
                   ├── name: str
                   └── arguments: str  # JSON string!

4. PARSING FLOW:
   Raw JSON → Pydantic models → Python objects

5. THE CRITICAL POINT:
   tool_call.function.arguments is intentionally kept as a string
   This allows the library to be agnostic about argument types
    
